In [ ]:
import numpy as np
import numpy.typing as npt
import random
import math
import matplotlib.pyplot as plt

from scipy.stats import norm
from sklearn.gaussian_process import GaussianProcessRegressor

In [ ]:
def sample(x):
    return x.reshape(-1, 1)

In [ ]:
"""
Parameters:
x (npt.NDArray): values are in range [-1.5, 7]
noise (float): random bias to add to the objective function

Returns:
npt.NDArray: y values of objective function calculated from given x
"""
def objective(x: npt.NDArray, noise = 1.5):
    x_squared = x * x
    x_cubed = x_squared * x
    bias = np.random.uniform(low=-noise, high=+noise, size=x.size) if noise is not None else 0
    return (0.25 * x_squared * np.cos(0.2 * x_squared)) - ((1 / 11) * np.exp(-x_cubed)) + 2 + bias

In [ ]:
"""
Parameters:
model (GaussianProcessRegressor): Gaussian Process
x (npt.NDArray): values in range [-1.5, 7]

Returns:
tuple[npt.NDArray, npt.NDArray]: predictions and standard deviation of predictions
"""
def surrogate(model, x):
    mu, std = model.predict(sample(x), return_std=True)
    return mu, std

In [ ]:
"""
Returns scores for each random variable

Parameters:
X (npt.NDArray): best random variables at moment Ti

Returns:
int: scores for random variables
"""
def acquisition(model, x, x_hat):
    mu, _ = surrogate(model, x)
    best_mu = np.max(mu)
    mu_hat, std_hat = surrogate(model, x_hat)
    scores = norm.cdf((mu_hat - best_mu) / std_hat)
    return scores

In [ ]:
def plot_input_variables(x, y, x_hat, y_hat):
    plt.plot(x, y, label="Objective function", color="blue")
    plt.scatter(x_hat, y_hat, label="Real input variables", color="red")
    plt.title("Objective vs Input")
    plt.legend()
    plt.show()

In [ ]:
def select(model, x):
    x_hat = np.asarray(np.random.uniform(low=-1.5, high=7, size=50))
    scores = acquisition(model, x, x_hat)
    ix = np.argmax(scores)
    return x_hat[ix]

In [ ]:
def plot_model_variables(x, y, model):
    x_hat = np.asarray(np.arange(-1.5, 7, 0.1))
    y_hat, _ = surrogate(model, x_hat)
    plt.plot(x, y, label="Objective function", color="blue")
    plt.plot(x_hat, y_hat, label="Model function", color="red")
    plt.title("Objective vs Model")
    plt.legend()
    plt.show()

In [ ]:
x = np.asarray(np.arange(-1.5, 7, 0.1))
x_hat = np.asarray(np.random.uniform(low=-1.5, high=7, size=15))

y = objective(x, noise=None)
y_hat = objective(x_hat)

In [ ]:
model = GaussianProcessRegressor()
model.fit(sample(x_hat), y_hat)

In [ ]:
def analyze(x, y, x_hat, y_hat, model):
    plot_input_variables(x, y, x_hat, y_hat)
    ix = np.argmax(y_hat)
    print('Best Result: x=%.3f, y=%.3f' % (x_hat[ix], y_hat[ix]))
    plot_model_variables(x, y, model)

In [ ]:
analyze(x, y, x_hat, y_hat, model)

In [ ]:
for i in range(100):
    x_best = select(model, x_hat)
    y_best = objective(x_best)
    
    mu_best, std_best = surrogate(model, np.array([x_best]))
    if i % 5 == 0:
        print(f"best sample at iteration {i}: x={x_best}, y={y_best}, mu={mu_best[0]}, std={std_best[0]}")
    
    x_hat = np.append(x_hat, x_best)
    y_hat = np.append(y_hat, y_best)
    
    model.fit(sample(x_hat), y_hat)

In [ ]:
analyze(x, y, x_hat, y_hat, model)